In [2]:
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
import napari

In [ ]:
%gui qt 
viewer = napari.Viewer()

In [5]:
impath = './220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Cntrl_E01-Z01_PI'
im = imread(impath + '.tif')
viewer.add_image(im, name='original', blending='additive')

<Image layer 'original' at 0x1ae71f070>

In [ ]:
from skimage import filters

for x in [1,3,6,9]:
    for y in [3,6,9,12,16]:
        if x<y:
            print(f'low = {x}, high = {y}')
            filtered = filters.difference_of_gaussians(im, low_sigma=x, high_sigma=y)
            print('finished filtering')
            viewer.add_image(filtered, name=f'{x}-{y}')
            print(f'added image {x}-{y} to viewer')

In [18]:
bigdog = filters.difference_of_gaussians(im, low_sigma=1, high_sigma=32)
viewer.add_image(bigdog, name='bigdog')

<Image layer 'bigdog' at 0x1b6f67c40>

In [19]:
bigbigdog = filters.difference_of_gaussians(im, low_sigma=1, high_sigma=64)
viewer.add_image(bigbigdog, name='bigbigdog')


<Image layer 'bigbigdog' at 0x1b852a340>

In [20]:
mediumbigdog = filters.difference_of_gaussians(im, low_sigma=2, high_sigma=64)
viewer.add_image(mediumbigdog, name='mediumbigdog')

<Image layer 'mediumbigdog' at 0x1b83c9400>

dog 1-16 seems okay

let's try log filtering first

In [23]:
logged = np.log2(im)
viewer.add_image(logged, name='logged2')

<Image layer 'logged2' at 0x1b6d9a160>

In [25]:
log_dog_2_64 = filters.difference_of_gaussians(logged, low_sigma=2, high_sigma=64)
log_dog_2_32 = filters.difference_of_gaussians(logged, low_sigma=2, high_sigma=32)
viewer.add_image(log_dog_2_64, name='log_dog_2_64')
viewer.add_image(log_dog_2_32, name='log_dog_2_32')


<Image layer 'log_dog_2_32' at 0x1b6e26cd0>

In [26]:
log_dog_1_16 = filters.difference_of_gaussians(logged, low_sigma=1, high_sigma=16)
viewer.add_image(log_dog_1_16, name='log_dog_1_16')

<Image layer 'log_dog_1_16' at 0x1b6f752b0>

medium big dog still the best

lapplacian of gaussian


In [34]:
import scipy
floated_im = im.astype(np.float32)
log = scipy.ndimage.gaussian_laplace(floated_im, sigma=6)
viewer.add_image(log, name='log_6')

<Image layer 'log_6' at 0x1b8572400>

In [35]:
sobel_dog = filters.sobel(mediumbigdog)
viewer.add_image(sobel_dog, name='sobel_dog')

<Image layer 'sobel_dog' at 0x1b8731670>

In [36]:
# otsu threahold medium big dog
otsu = mediumbigdog > filters.threshold_otsu(mediumbigdog)
viewer.add_image(otsu, name='otsu')


<Image layer 'otsu' at 0x1b8731340>

In [37]:
scharr = filters.scharr(im)
viewer.add_image(scharr, name='scharr')

<Image layer 'scharr' at 0x1b6e1b0a0>

In [40]:
distance = ndimage.distance_transform_edt(mediumbigdog)
viewer.add_image(distance, name='distance')

<Image layer 'distance' at 0x1b8690fd0>

In [41]:
superbigdog_low1 = filters.difference_of_gaussians(im, low_sigma=1, high_sigma=128)
superbigdog_low2 = filters.difference_of_gaussians(im, low_sigma=2, high_sigma=128)
viewer.add_image(superbigdog_low1, name='superbigdog_low1')
viewer.add_image(superbigdog_low2, name='superbigdog_low2')

<Image layer 'superbigdog_low2' at 0x1b8468f40>

In [44]:
highpass = im - filters.gaussian(im, sigma=3)
viewer.add_image(highpass, name='highpass')

<Image layer 'highpass' at 0x1b6f81bb0>

In [47]:
highpassdog = mediumbigdog - filters.gaussian(mediumbigdog, sigma=3)
viewer.add_image(mediumbigdog, name='mediumbigdog')
viewer.add_image(highpassdog, name='highpassdog')

/Users/bementmbp/opt/anaconda3/envs/image-analysis/lib/python3.9/site-packages/vispy/visuals/shaders/function.py:485: RuntimeWarning: coroutine 'get_msg' was never awaited
  min_indent = min(indent, min_indent)


<Image layer 'highpassdog' at 0x1b87e59d0>

batch process all ims with superbig dog low 2

In [45]:
import os
from tqdm import tqdm
from skimage import filters
from tifffile import imread, imwrite

processing_dir = '/Volumes/bigData/wholeMount_volDist/220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Processed/N2V_Denoised/16bit_scaleZ'
save_dir = os.path.join(processing_dir, 'sbdl2_processed')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

im_names = [f for f in os.listdir(processing_dir) if f.endswith('.tif') and not f.startswith('.')]

with tqdm(total=len(im_names)) as pbar:
    for im_name in im_names:
        im_base = im_name.split('.')[0]
        im = imread(os.path.join(processing_dir, im_name))
        sbdl2 = filters.difference_of_gaussians(im, low_sigma=2, high_sigma=128)
        imwrite(os.path.join(save_dir, f'{im_base}_sbdl2.tif'), sbdl2)
        pbar.update(1)


100%|██████████| 32/32 [2:29:23<00:00, 280.12s/it]  
